# Setup

In [1]:
import sys
import os
madminer_src_path = "/home/shomiller/madminer"
sys.path.append(madminer_src_path)

from __future__ import absolute_import, division, print_function, unicode_literals
import logging

import numpy as np
import math
import matplotlib
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
% matplotlib inline
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

from madminer.fisherinformation import FisherInformation
from madminer.fisherinformation import project_information, profile_information

from pandas import DataFrame

import madminer.__version__
print( 'MadMiner version: {}'.format(madminer.__version__) )

MadMiner version: 0.4.9


In [2]:
# MadMiner output
logging.basicConfig(
    format='%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s',
    datefmt='%H:%M',
    level=logging.INFO
)

# Output of all other modules (e.g. matplotlib)
for key in logging.Logger.manager.loggerDict:
    if "madminer" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

In [3]:
uselumi=300.

In [4]:
labels = [ 'cHbox', 'cHDD', 'cHW', 'cHq3' ]
labels_rot = [ 'cHK', 'cHW', 'cHq3' ]

### Data Files:

Saving the paths for all our lhe data files, for easy use later on.

Note, lhedata are saved in the format:
`data/{channel}_lhedata_{observables}.h5`
while the `SALLY` models are saved in:
`models/sally_ensemble_{channel}_{observables}/`

here `channel` can be e.g, `wph_mu_wbkgs` or `wmh_e_wbkgs` (or take out the backgrounds for the background free runs), while `observables` can be `full`, `met`, `ptw` or `2d`

#### Signal + Background with Signal Systematics Only

In [5]:
lhedatafile_wph_mu_wbkgs_met = 'data/met/wph_mu_wbkgs_sigsystonly_lhedata_met.h5'
lhedatafile_wph_e_wbkgs_met = 'data/met/wph_e_wbkgs_sigsystonly_lhedata_met.h5'
lhedatafile_wmh_mu_wbkgs_met = 'data/met/wmh_mu_wbkgs_sigsystonly_lhedata_met.h5'
lhedatafile_wmh_e_wbkgs_met = 'data/met/wmh_e_wbkgs_sigsystonly_lhedata_met.h5'

## Setup FisherInfo Objects

In [6]:
fisher_wph_mu_wbkgs_met = FisherInformation(lhedatafile_wph_mu_wbkgs_met, include_nuisance_parameters=True)

13:30 madminer.analysis    INFO    Loading data from data/met/wph_mu_wbkgs_sigsystonly_lhedata_met.h5
13:30 madminer.analysis    INFO    Found 4 parameters
13:30 madminer.analysis    INFO    Found 33 nuisance parameters
13:30 madminer.analysis    INFO    Found 54 benchmarks, of which 15 physical
13:30 madminer.analysis    INFO    Found 48 observables
13:30 madminer.analysis    INFO    Found 5000000 events
13:30 madminer.analysis    INFO      1000000 signal events sampled from benchmark sm
13:30 madminer.analysis    INFO      4000000 background events
13:30 madminer.analysis    INFO    Found morphing setup with 15 components
13:30 madminer.analysis    INFO    Found nuisance morphing setup


In [7]:
fisher_wph_e_wbkgs_met = FisherInformation(lhedatafile_wph_e_wbkgs_met, include_nuisance_parameters=True)

13:30 madminer.analysis    INFO    Loading data from data/met/wph_e_wbkgs_sigsystonly_lhedata_met.h5
13:30 madminer.analysis    INFO    Found 4 parameters
13:30 madminer.analysis    INFO    Found 33 nuisance parameters
13:30 madminer.analysis    INFO    Found 54 benchmarks, of which 15 physical
13:30 madminer.analysis    INFO    Found 48 observables
13:30 madminer.analysis    INFO    Found 5000000 events
13:30 madminer.analysis    INFO      1000000 signal events sampled from benchmark sm
13:30 madminer.analysis    INFO      4000000 background events
13:30 madminer.analysis    INFO    Found morphing setup with 15 components
13:30 madminer.analysis    INFO    Found nuisance morphing setup


In [8]:
fisher_wmh_mu_wbkgs_met = FisherInformation(lhedatafile_wmh_mu_wbkgs_met, include_nuisance_parameters=True)

13:30 madminer.analysis    INFO    Loading data from data/met/wmh_mu_wbkgs_sigsystonly_lhedata_met.h5
13:30 madminer.analysis    INFO    Found 4 parameters
13:30 madminer.analysis    INFO    Found 33 nuisance parameters
13:30 madminer.analysis    INFO    Found 54 benchmarks, of which 15 physical
13:30 madminer.analysis    INFO    Found 48 observables
13:30 madminer.analysis    INFO    Found 5000000 events
13:30 madminer.analysis    INFO      1000000 signal events sampled from benchmark sm
13:30 madminer.analysis    INFO      4000000 background events
13:30 madminer.analysis    INFO    Found morphing setup with 15 components
13:30 madminer.analysis    INFO    Found nuisance morphing setup


In [9]:
fisher_wmh_e_wbkgs_met = FisherInformation(lhedatafile_wmh_e_wbkgs_met, include_nuisance_parameters=True)

13:30 madminer.analysis    INFO    Loading data from data/met/wmh_e_wbkgs_sigsystonly_lhedata_met.h5
13:30 madminer.analysis    INFO    Found 4 parameters
13:30 madminer.analysis    INFO    Found 33 nuisance parameters
13:30 madminer.analysis    INFO    Found 54 benchmarks, of which 15 physical
13:30 madminer.analysis    INFO    Found 48 observables
13:30 madminer.analysis    INFO    Found 5000000 events
13:30 madminer.analysis    INFO      1000000 signal events sampled from benchmark sm
13:30 madminer.analysis    INFO      4000000 background events
13:30 madminer.analysis    INFO    Found morphing setup with 15 components
13:30 madminer.analysis    INFO    Found nuisance morphing setup


# Information in Histograms

## 1D Histograms

### Simplified Template Cross Sections

The Info in the STXS defined in https://cds.cern.ch/record/2669925 (with and without the extra bin)

#### 3 Bins
The original STXS defined in The LHC Higgs Cross Section Working Group Report, Vol. 4 (https://arxiv.org/abs/1610.07922)

In [10]:
fi_stxs_3bins_wph_mu_mean, fi_stxs_3bins_wph_mu_covariance = fisher_wph_mu_wbkgs_met.calculate_fisher_information_hist1d(
    theta=[0.,0.,0.,0.],
    luminosity=uselumi*1000.,
    observable='pt_w',
    bins=[0.,150.,250.],
    histrange=[0.,250.]
)

print( DataFrame(
    data = project_information(fi_stxs_3bins_wph_mu_mean, [0,1,2,3]),
    columns = labels,
    index = labels)
)

13:40 madminer.fisherinfor INFO    Bins with largest statistical uncertainties on rates:
13:40 madminer.fisherinfor INFO      Bin 4: (2.00747 +/- 0.00709) fb (0 %)
13:40 madminer.fisherinfor INFO      Bin 3: (9.07794 +/- 0.01585) fb (0 %)
13:40 madminer.fisherinfor INFO      Bin 2: (128.65914 +/- 0.06208) fb (0 %)
13:40 madminer.fisherinfor INFO      Bin 1: (0.00000 +/- 0.00000) fb (0 %)


           cHbox      cHDD        cHW      cHq3
cHbox  30.782566 -7.695640  11.738873  5.159941
cHDD   -7.695640  1.923910  -2.934718 -1.289985
cHW    11.738873 -2.934718   4.519365  2.129346
cHq3    5.159941 -1.289985   2.129346  1.637871


/home/shomiller/.conda/envs/my_root/lib/python2.7/site-packages/madminer-0.4.9-py2.7.egg/madminer/fisherinformation.py:1166: RuntimeWarning: divide by zero encountered in true_divide


In [11]:
fi_stxs_3bins_wph_e_mean, fi_stxs_3bins_wph_e_covariance = fisher_wph_e_wbkgs_met.calculate_fisher_information_hist1d(
    theta=[0.,0.,0.,0.],
    luminosity=uselumi*1000.,
    observable='pt_w',
    bins=[0.,150.,250.],
    histrange=[0.,250.]
)

13:51 madminer.fisherinfor INFO    Bins with largest statistical uncertainties on rates:
13:51 madminer.fisherinfor INFO      Bin 4: (2.01261 +/- 0.00709) fb (0 %)
13:51 madminer.fisherinfor INFO      Bin 3: (9.06235 +/- 0.01583) fb (0 %)
13:51 madminer.fisherinfor INFO      Bin 2: (128.66962 +/- 0.06208) fb (0 %)
13:51 madminer.fisherinfor INFO      Bin 1: (0.00000 +/- 0.00000) fb (0 %)


In [12]:
fi_stxs_3bins_wmh_mu_mean, fi_stxs_3bins_wmh_mu_covariance = fisher_wmh_mu_wbkgs_met.calculate_fisher_information_hist1d(
    theta=[0.,0.,0.,0.],
    luminosity=uselumi*1000.,
    observable='pt_w',
    bins=[0.,150.,250.],
    histrange=[0.,250.]
)

14:02 madminer.fisherinfor INFO    Bins with largest statistical uncertainties on rates:
14:02 madminer.fisherinfor INFO      Bin 4: (1.04309 +/- 0.00423) fb (0 %)
14:02 madminer.fisherinfor INFO      Bin 3: (6.40072 +/- 0.01197) fb (0 %)
14:02 madminer.fisherinfor INFO      Bin 2: (97.02838 +/- 0.04738) fb (0 %)
14:02 madminer.fisherinfor INFO      Bin 1: (0.00000 +/- 0.00000) fb (0 %)


In [13]:
fi_stxs_3bins_wmh_e_mean, fi_stxs_3bins_wmh_e_covariance = fisher_wmh_e_wbkgs_met.calculate_fisher_information_hist1d(
    theta=[0.,0.,0.,0.],
    luminosity=uselumi*1000.,
    observable='pt_w',
    bins=[0.,150.,250.],
    histrange=[0.,250.]
)

14:12 madminer.fisherinfor INFO    Bins with largest statistical uncertainties on rates:
14:12 madminer.fisherinfor INFO      Bin 4: (1.04329 +/- 0.00423) fb (0 %)
14:12 madminer.fisherinfor INFO      Bin 3: (6.39363 +/- 0.01196) fb (0 %)
14:12 madminer.fisherinfor INFO      Bin 2: (97.03527 +/- 0.04738) fb (0 %)
14:12 madminer.fisherinfor INFO      Bin 1: (0.00000 +/- 0.00000) fb (0 %)


In [14]:
fi_stxs_3bins_mean = fi_stxs_3bins_wph_mu_mean + fi_stxs_3bins_wph_e_mean + fi_stxs_3bins_wmh_mu_mean + fi_stxs_3bins_wmh_e_mean
fi_stxs_3bins_covariance = fi_stxs_3bins_wph_mu_covariance + fi_stxs_3bins_wph_e_covariance + fi_stxs_3bins_wmh_mu_covariance + fi_stxs_3bins_wmh_e_covariance

#### 4 Bins 
(Including the Extra Division at $75\,\mathrm{GeV}$ as in https://cds.cern.ch/record/2669925)

(We're not computing this for now, just setting the values to zero as placeholders)

In [15]:
fi_stxs_4bins_wph_mu_mean, fi_stxs_4bins_wph_mu_covariance = 0., 0.
fi_stxs_4bins_wph_e_mean, fi_stxs_4bins_wph_e_covariance = 0., 0.
fi_stxs_4bins_wmh_mu_mean, fi_stxs_4bins_wmh_mu_covariance = 0., 0.
fi_stxs_4bins_wmh_e_mean, fi_stxs_4bins_wmh_e_covariance = 0., 0.

#### 5 Bins
Including all the bins in the Stage 1.1 STXS defined in https://cds.cern.ch/record/2669925

Beyond the bins at 0, 150, and 250 GeV, this includes additional bin divisions at $75$ and $400\,\mathrm{GeV}$.

In [16]:
fi_stxs_5bins_wph_mu_mean, fi_stxs_5bins_wph_mu_covariance = fisher_wph_mu_wbkgs_met.calculate_fisher_information_hist1d(
    theta=[0.,0.,0.,0.],
    luminosity=uselumi*1000.,
    observable='pt_w',
    bins=[0.,75.,150.,250.,400.],
    histrange=[0.,400.]
)

print( DataFrame(
    data = project_information(fi_stxs_5bins_wph_mu_mean, [0,1,2,3]),
    columns = labels,
    index = labels)
)

14:23 madminer.fisherinfor INFO    Bins with largest statistical uncertainties on rates:
14:23 madminer.fisherinfor INFO      Bin 6: (0.35838 +/- 0.00293) fb (1 %)
14:23 madminer.fisherinfor INFO      Bin 5: (1.64909 +/- 0.00645) fb (0 %)
14:23 madminer.fisherinfor INFO      Bin 4: (9.07794 +/- 0.01585) fb (0 %)
14:23 madminer.fisherinfor INFO      Bin 3: (48.03024 +/- 0.03754) fb (0 %)
14:23 madminer.fisherinfor INFO      Bin 2: (80.62890 +/- 0.04944) fb (0 %)


           cHbox      cHDD        cHW      cHq3
cHbox  30.917945 -7.729485  11.817753  5.292626
cHDD   -7.729485  1.932371  -2.954438 -1.323156
cHW    11.817753 -2.954438   4.565517  2.199306
cHq3    5.292626 -1.323156   2.199306  2.047863


In [17]:
fi_stxs_5bins_wph_e_mean, fi_stxs_5bins_wph_e_covariance = fisher_wph_e_wbkgs_met.calculate_fisher_information_hist1d(
    theta=[0.,0.,0.,0.],
    luminosity=uselumi*1000.,
    observable='pt_w',
    bins=[0.,75.,150.,250.,400.],
    histrange=[0.,400.]
)

14:32 madminer.fisherinfor INFO    Bins with largest statistical uncertainties on rates:
14:32 madminer.fisherinfor INFO      Bin 6: (0.35715 +/- 0.00293) fb (1 %)
14:32 madminer.fisherinfor INFO      Bin 5: (1.65546 +/- 0.00646) fb (0 %)
14:32 madminer.fisherinfor INFO      Bin 4: (9.06235 +/- 0.01583) fb (0 %)
14:32 madminer.fisherinfor INFO      Bin 3: (48.06845 +/- 0.03756) fb (0 %)
14:32 madminer.fisherinfor INFO      Bin 2: (80.60117 +/- 0.04943) fb (0 %)


In [18]:
fi_stxs_5bins_wmh_mu_mean, fi_stxs_5bins_wmh_mu_covariance = fisher_wmh_mu_wbkgs_met.calculate_fisher_information_hist1d(
    theta=[0.,0.,0.,0.],
    luminosity=uselumi*1000.,
    observable='pt_w',
    bins=[0.,75.,150.,250.,400.],
    histrange=[0.,400.]
)

14:43 madminer.fisherinfor INFO    Bins with largest statistical uncertainties on rates:
14:43 madminer.fisherinfor INFO      Bin 6: (0.16106 +/- 0.00161) fb (1 %)
14:43 madminer.fisherinfor INFO      Bin 5: (0.88203 +/- 0.00391) fb (0 %)
14:43 madminer.fisherinfor INFO      Bin 4: (6.40072 +/- 0.01197) fb (0 %)
14:43 madminer.fisherinfor INFO      Bin 3: (37.14346 +/- 0.02997) fb (0 %)
14:43 madminer.fisherinfor INFO      Bin 2: (59.88492 +/- 0.03670) fb (0 %)


In [19]:
fi_stxs_5bins_wmh_e_mean, fi_stxs_5bins_wmh_e_covariance = fisher_wmh_e_wbkgs_met.calculate_fisher_information_hist1d(
    theta=[0.,0.,0.,0.],
    luminosity=uselumi*1000.,
    observable='pt_w',
    bins=[0.,75.,150.,250.,400.],
    histrange=[0.,400.]
)

14:53 madminer.fisherinfor INFO    Bins with largest statistical uncertainties on rates:
14:53 madminer.fisherinfor INFO      Bin 6: (0.15967 +/- 0.00161) fb (1 %)
14:53 madminer.fisherinfor INFO      Bin 5: (0.88362 +/- 0.00392) fb (0 %)
14:53 madminer.fisherinfor INFO      Bin 4: (6.39363 +/- 0.01196) fb (0 %)
14:53 madminer.fisherinfor INFO      Bin 3: (37.16186 +/- 0.02997) fb (0 %)
14:53 madminer.fisherinfor INFO      Bin 2: (59.87340 +/- 0.03670) fb (0 %)


In [20]:
fi_stxs_5bins_mean = fi_stxs_5bins_wph_mu_mean + fi_stxs_5bins_wph_e_mean + fi_stxs_5bins_wmh_mu_mean + fi_stxs_5bins_wmh_e_mean
fi_stxs_5bins_covariance = fi_stxs_5bins_wph_mu_covariance + fi_stxs_5bins_wph_e_covariance + fi_stxs_5bins_wmh_mu_covariance + fi_stxs_5bins_wmh_e_covariance

#### 6 Bins
As a step towards "Improved STXS", we can also include an additional bin at $600\,\mathrm{GeV}$

In [21]:
fi_stxs_6bins_wph_mu_mean, fi_stxs_6bins_wph_mu_covariance = fisher_wph_mu_wbkgs_met.calculate_fisher_information_hist1d(
    theta=[0.,0.,0.,0.],
    luminosity=uselumi*1000.,
    observable='pt_w',
    bins=[0.,75.,150.,250.,400.,600.],
    histrange=[0.,600.]
)

print( DataFrame(
    data = project_information(fi_stxs_6bins_wph_mu_mean, [0,1,2,3]),
    columns = labels,
    index = labels)
)

15:04 madminer.fisherinfor INFO    Bins with largest statistical uncertainties on rates:
15:04 madminer.fisherinfor INFO      Bin 7: (0.03894 +/- 0.00095) fb (2 %)
15:04 madminer.fisherinfor INFO      Bin 6: (0.31944 +/- 0.00277) fb (1 %)
15:04 madminer.fisherinfor INFO      Bin 5: (1.64909 +/- 0.00645) fb (0 %)
15:04 madminer.fisherinfor INFO      Bin 4: (9.07794 +/- 0.01585) fb (0 %)
15:04 madminer.fisherinfor INFO      Bin 3: (48.03024 +/- 0.03754) fb (0 %)


           cHbox      cHDD        cHW      cHq3
cHbox  30.918957 -7.729738  11.818467  5.303187
cHDD   -7.729738  1.932434  -2.954616 -1.325796
cHW    11.818467 -2.954616   4.566021  2.206757
cHq3    5.303187 -1.325796   2.206757  2.158055


In [22]:
fi_stxs_6bins_wph_e_mean, fi_stxs_6bins_wph_e_covariance = fisher_wph_e_wbkgs_met.calculate_fisher_information_hist1d(
    theta=[0.,0.,0.,0.],
    luminosity=uselumi*1000.,
    observable='pt_w',
    bins=[0.,75.,150.,250.,400.,600.],
    histrange=[0.,600.]
)

15:14 madminer.fisherinfor INFO    Bins with largest statistical uncertainties on rates:
15:14 madminer.fisherinfor INFO      Bin 7: (0.03818 +/- 0.00094) fb (2 %)
15:14 madminer.fisherinfor INFO      Bin 6: (0.31897 +/- 0.00277) fb (1 %)
15:14 madminer.fisherinfor INFO      Bin 5: (1.65546 +/- 0.00646) fb (0 %)
15:14 madminer.fisherinfor INFO      Bin 4: (9.06235 +/- 0.01583) fb (0 %)
15:14 madminer.fisherinfor INFO      Bin 3: (48.06845 +/- 0.03756) fb (0 %)


In [23]:
fi_stxs_6bins_wmh_mu_mean, fi_stxs_6bins_wmh_mu_covariance = fisher_wmh_mu_wbkgs_met.calculate_fisher_information_hist1d(
    theta=[0.,0.,0.,0.],
    luminosity=uselumi*1000.,
    observable='pt_w',
    bins=[0.,75.,150.,250.,400.,600.],
    histrange=[0.,600.]
)

15:24 madminer.fisherinfor INFO    Bins with largest statistical uncertainties on rates:
15:24 madminer.fisherinfor INFO      Bin 7: (0.01469 +/- 0.00047) fb (3 %)
15:24 madminer.fisherinfor INFO      Bin 6: (0.14638 +/- 0.00154) fb (1 %)
15:24 madminer.fisherinfor INFO      Bin 5: (0.88203 +/- 0.00391) fb (0 %)
15:24 madminer.fisherinfor INFO      Bin 4: (6.40072 +/- 0.01197) fb (0 %)
15:24 madminer.fisherinfor INFO      Bin 3: (37.14346 +/- 0.02997) fb (0 %)


In [24]:
fi_stxs_6bins_wmh_e_mean, fi_stxs_6bins_wmh_e_covariance = fisher_wmh_e_wbkgs_met.calculate_fisher_information_hist1d(
    theta=[0.,0.,0.,0.],
    luminosity=uselumi*1000.,
    observable='pt_w',
    bins=[0.,75.,150.,250.,400.,600.],
    histrange=[0.,600.]
)

15:34 madminer.fisherinfor INFO    Bins with largest statistical uncertainties on rates:
15:34 madminer.fisherinfor INFO      Bin 7: (0.01431 +/- 0.00046) fb (3 %)
15:34 madminer.fisherinfor INFO      Bin 6: (0.14536 +/- 0.00154) fb (1 %)
15:34 madminer.fisherinfor INFO      Bin 5: (0.88362 +/- 0.00392) fb (0 %)
15:34 madminer.fisherinfor INFO      Bin 4: (6.39363 +/- 0.01196) fb (0 %)
15:34 madminer.fisherinfor INFO      Bin 3: (37.16186 +/- 0.02997) fb (0 %)


In [25]:
fi_stxs_6bins_mean = fi_stxs_6bins_wph_mu_mean + fi_stxs_6bins_wph_e_mean + fi_stxs_6bins_wmh_mu_mean + fi_stxs_6bins_wmh_e_mean
fi_stxs_6bins_covariance = fi_stxs_6bins_wph_mu_covariance + fi_stxs_6bins_wph_e_covariance + fi_stxs_6bins_wmh_mu_covariance + fi_stxs_6bins_wmh_e_covariance

### Save the Arrays for Later

In [26]:
fi_hist_stxs_outfile = './fisher_info/fi_hist_stxs.npz'
fi_hist_stxs_cov_outfile = './fisher_info/fi_hist_stxs_cov.npz'

fi_hist_stxs_list = [
    fi_stxs_3bins_wph_mu_mean,
    fi_stxs_3bins_wph_e_mean,
    fi_stxs_3bins_wmh_mu_mean,
    fi_stxs_3bins_wmh_e_mean,
    fi_stxs_4bins_wph_mu_mean,
    fi_stxs_4bins_wph_e_mean,
    fi_stxs_4bins_wmh_mu_mean,
    fi_stxs_4bins_wmh_e_mean,
    fi_stxs_5bins_wph_mu_mean,
    fi_stxs_5bins_wph_e_mean,
    fi_stxs_5bins_wmh_mu_mean,
    fi_stxs_5bins_wmh_e_mean,
    fi_stxs_6bins_wph_mu_mean,
    fi_stxs_6bins_wph_e_mean,
    fi_stxs_6bins_wmh_mu_mean,
    fi_stxs_6bins_wmh_e_mean,
]

fi_hist_stxs_cov_list = [
    fi_stxs_3bins_wph_mu_covariance,
    fi_stxs_3bins_wph_e_covariance,
    fi_stxs_3bins_wmh_mu_covariance,
    fi_stxs_3bins_wmh_e_covariance,
    fi_stxs_4bins_wph_mu_covariance,
    fi_stxs_4bins_wph_e_covariance,
    fi_stxs_4bins_wmh_mu_covariance,
    fi_stxs_4bins_wmh_e_covariance,
    fi_stxs_5bins_wph_mu_covariance,
    fi_stxs_5bins_wph_e_covariance,
    fi_stxs_5bins_wmh_mu_covariance,
    fi_stxs_5bins_wmh_e_covariance,
    fi_stxs_6bins_wph_mu_covariance,
    fi_stxs_6bins_wph_e_covariance,
    fi_stxs_6bins_wmh_mu_covariance,
    fi_stxs_6bins_wmh_e_covariance,
]

In [27]:
np.savez(fi_hist_stxs_outfile, fi_hist_stxs_list, allow_pickle=False)
np.savez(fi_hist_stxs_cov_outfile, fi_hist_stxs_cov_list, allow_pickle=False)

## 2D Histograms

### Compute FI in 2D Histograms

#### With 3 bins in $m_{T,\mathrm{tot}}$

In [28]:
fi_imp_stxs_wph_mu_mean, fi_imp_stxs_wph_mu_covariance = fisher_wph_mu_wbkgs_met.calculate_fisher_information_hist2d(
    theta=[0.,0.,0.,0.],
    luminosity=uselumi*1000.,
    observable1='pt_w',
    bins1=[0.,75.,150.,250.,400.,600.],
    histrange1=[0.,600.],
    observable2='mt_tot',
    bins2=[0.,400.,800.],
    histrange2=[0.,800.],
)

15:49 madminer.fisherinfor INFO    Bins with largest statistical uncertainties on rates:
15:49 madminer.fisherinfor INFO      Bin (2, 1): (0.00449 +/- 0.00038) fb (9 %)
15:49 madminer.fisherinfor INFO      Bin (2, 5): (0.01418 +/- 0.00068) fb (5 %)
15:49 madminer.fisherinfor INFO      Bin (4, 7): (0.03894 +/- 0.00095) fb (2 %)
15:49 madminer.fisherinfor INFO      Bin (3, 2): (0.06982 +/- 0.00147) fb (2 %)
15:49 madminer.fisherinfor INFO      Bin (4, 3): (0.31944 +/- 0.00277) fb (1 %)


In [29]:
fi_imp_stxs_wph_e_mean, fi_imp_stxs_wph_e_covariance = fisher_wph_e_wbkgs_met.calculate_fisher_information_hist2d(
    theta=[0.,0.,0.,0.],
    luminosity=uselumi*1000.,
    observable1='pt_w',
    bins1=[0.,75.,150.,250.,400.,600.],
    histrange1=[0.,600.],
    observable2='mt_tot',
    bins2=[0.,400.,800.],
    histrange2=[0.,800.],
)

16:03 madminer.fisherinfor INFO    Bins with largest statistical uncertainties on rates:
16:03 madminer.fisherinfor INFO      Bin (4, 2): (0.00001 +/- 0.00001) fb (100 %)
16:03 madminer.fisherinfor INFO      Bin (2, 1): (0.00447 +/- 0.00038) fb (9 %)
16:03 madminer.fisherinfor INFO      Bin (2, 5): (0.01450 +/- 0.00069) fb (5 %)
16:03 madminer.fisherinfor INFO      Bin (4, 7): (0.03818 +/- 0.00094) fb (2 %)
16:03 madminer.fisherinfor INFO      Bin (3, 2): (0.06938 +/- 0.00147) fb (2 %)


In [30]:
fi_imp_stxs_wmh_mu_mean, fi_imp_stxs_wmh_mu_covariance = fisher_wmh_mu_wbkgs_met.calculate_fisher_information_hist2d(
    theta=[0.,0.,0.,0.],
    luminosity=uselumi*1000.,
    observable1='pt_w',
    bins1=[0.,75.,150.,250.,400.,600.],
    histrange1=[0.,600.],
    observable2='mt_tot',
    bins2=[0.,400.,800.],
    histrange2=[0.,800.],
)

16:17 madminer.fisherinfor INFO    Bins with largest statistical uncertainties on rates:
16:17 madminer.fisherinfor INFO      Bin (2, 1): (0.00208 +/- 0.00022) fb (10 %)
16:17 madminer.fisherinfor INFO      Bin (2, 5): (0.00685 +/- 0.00039) fb (6 %)
16:17 madminer.fisherinfor INFO      Bin (4, 7): (0.01469 +/- 0.00047) fb (3 %)
16:17 madminer.fisherinfor INFO      Bin (3, 2): (0.03802 +/- 0.00090) fb (2 %)
16:17 madminer.fisherinfor INFO      Bin (4, 3): (0.14638 +/- 0.00154) fb (1 %)


In [31]:
fi_imp_stxs_wmh_e_mean, fi_imp_stxs_wmh_e_covariance = fisher_wmh_e_wbkgs_met.calculate_fisher_information_hist2d(
    theta=[0.,0.,0.,0.],
    luminosity=uselumi*1000.,
    observable1='pt_w',
    bins1=[0.,75.,150.,250.,400.,600.],
    histrange1=[0.,600.],
    observable2='mt_tot',
    bins2=[0.,400.,800.],
    histrange2=[0.,800.],
)

16:31 madminer.fisherinfor INFO    Bins with largest statistical uncertainties on rates:
16:31 madminer.fisherinfor INFO      Bin (4, 2): (0.00002 +/- 0.00002) fb (100 %)
16:31 madminer.fisherinfor INFO      Bin (2, 1): (0.00200 +/- 0.00021) fb (11 %)
16:31 madminer.fisherinfor INFO      Bin (2, 5): (0.00680 +/- 0.00039) fb (6 %)
16:31 madminer.fisherinfor INFO      Bin (4, 7): (0.01431 +/- 0.00046) fb (3 %)
16:31 madminer.fisherinfor INFO      Bin (3, 2): (0.03874 +/- 0.00091) fb (2 %)


In [32]:
fi_imp_stxs_mean = fi_imp_stxs_wph_mu_mean + fi_imp_stxs_wph_e_mean + fi_imp_stxs_wmh_mu_mean + fi_imp_stxs_wmh_e_mean
fi_imp_stxs_covariance = fi_imp_stxs_wph_mu_covariance + fi_imp_stxs_wph_e_covariance + fi_imp_stxs_wmh_mu_covariance + fi_imp_stxs_wmh_e_covariance

### Save the Arrays for Later

In [33]:
fi_hist_2d_outfile = './fisher_info/fi_hist_2d.npz'
fi_hist_2d_cov_outfile = './fisher_info/fi_hist_2d_cov.npz'

fi_hist_2d_list = [
    fi_imp_stxs_wph_mu_mean, 
    fi_imp_stxs_wph_e_mean, 
    fi_imp_stxs_wmh_mu_mean, 
    fi_imp_stxs_wmh_e_mean, 
]

fi_hist_2d_cov_list = [
    fi_imp_stxs_wph_mu_covariance,
    fi_imp_stxs_wph_e_covariance,
    fi_imp_stxs_wmh_mu_covariance,
    fi_imp_stxs_wmh_e_covariance,
]

In [34]:
np.savez(fi_hist_2d_outfile, fi_hist_2d_list, allow_pickle=False)
np.savez(fi_hist_2d_cov_outfile, fi_hist_2d_cov_list, allow_pickle=False)